In [1]:
import os
os.chdir("../")

In [2]:
!pwd

/Users/sohamsamal/Desktop/Coding/Learn/MLOPs/chest-cancer-classifier


In [ ]:
# Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
# Configuration Manager

from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
# Components

import os
import zipfile
import gdown
from chestCancerClassifier import logger
from chestCancerClassifier.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
# Pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-23 23:58:52,827: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-23 23:58:52,829: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-23 23:58:52,830: INFO: common: created directory at: artifacts]
[2025-06-23 23:58:52,831: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-23 23:58:52,832: INFO: 1701442284: Downloading data from https://drive.google.com/file/d/1-VK4ODCVtuJcWDviMEohAVFI9JuSQCP9/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1-VK4ODCVtuJcWDviMEohAVFI9JuSQCP9
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1-VK4ODCVtuJcWDviMEohAVFI9JuSQCP9&confirm=t&uuid=2faa0963-9ff6-45ec-bd7d-438a7949b28d
To: /Users/sohamsamal/Desktop/Coding/Learn/MLOPs/chest-cancer-classifier/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:12<00:00, 10.2MB/s] 


[2025-06-23 23:59:07,935: INFO: 1701442284: Downloaded data from https://drive.google.com/file/d/1-VK4ODCVtuJcWDviMEohAVFI9JuSQCP9/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
